In [ ]:
# SOLID

In [ ]:
## S : 단일 책임 원칙
class SystemMonitor:
    def load_activity(self):
        """소스에서 처리할 이벤트 가져오기"""

    def identify_events(self):
        """가저온 데이터를 파싱하여 도메인 객체 이벤트로 변환"""

    def stream_events(self):
        """파싱한 이벤트를 외부 에이전트로 전송"""


"""
독립적인 동작을 하는 메서드를 하나의 인터페이스에 정의함.
    각 동작이 나머지 부분과 독립적으로 수행할 수 있음
"""

In [12]:
## O : 개방 / 폐쇄 원칙
"""
확장에는 개방되어 있으며, 
수정에는 폐쇄되어 있는 디자인
"""
from dataclasses import dataclass


################################################################
@dataclass
class Event:
    raw_data: dict


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    """로그인 사용자에 의한 이벤트"""


class LogoutEvent(Event):
    """로그아웃 사용자에 의한 이벤트"""


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data) -> None:
        self.event_data = event_data

    def identify_event(self):
        if (
            self.event_data["before"]["session"] == 0
            and self.event_data["after"]["session"] == 1
        ):
            return LoginEvent(self.event_data)

        elif (
            self.event_data["before"]["session"] == 1
            and self.event_data["after"]["session"] == 0
        ):
            return LogoutEvent(self.event_data)
        return UnknownEvent(self.event_data)


"""
문제 :
    이벤트 유형을 결정하는 로직이 단일 메서드에서 중앙 집중화 된다.
        지원하려는 이벤트가 늘어날수록 메서드도 커질 것이므로 결국 매우 큰 메서드가 될 것.
    메서드가 수정을 위해 닫히지 않았다.
        새로운 유형의 이벤트를 추가할 때마다 메서드를 수정해야한다.
그러므로
    1. 이 메서드를 변경하지 않고 새로운 유형의 이벤트를 추가하고 싶다 (폐쇄)
    2. 새로운 이벤트가 추가될 때 이미 존재하는 코드를 변경하지 않고 코드를 확장하여 지원하고 싶다 (개방)
"""


@dataclass
class Event:
    raw_data: dict

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return False


class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data) -> None:
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)


## 추가시
class TransactionEvent(Event):
    """시스템에서 발생한 트랜잭션 이벤트"""

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return event_data["after"].get("transaction") is not None

In [26]:
class LogoutEvent(Event):
    def meets_condition(self, event_data: dict, ovverride: bool) -> bool:
        if ovverride:
            return True
        pass


################################################################
from collections.abc import Mapping


class Event:
    def __init__(self, raw_data) -> None:
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict) -> bool:
        return False

    @staticmethod
    def validate_precondition(event_data: dict):
        """
        인터페이스 계약의 사전 조건
        ''event_data'' 파라미터가 적절한 형태인지 유효성 검사
        """
        if not isinstance(event_data, Mapping):
            raise ValueError(f"{event_data!r} dict 데이터 타입이 아님")
        for moment in ("before", "after"):
            if moment not in event_data:
                raise ValueError(f"{event_data} 에 {moment} 정보 없음")
            if not isinstance(event_data[moment], Mapping):
                raise ValueError(f"even_data[{moment!r}] dict 데이터 타입 아님!")


class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""

    def __init__(self, event_data) -> None:
        self.even_data = event_data

    def identify_event(self):
        Event.validate_precondition(self.even_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition(self.even_data)
            ),
            UnknownEvent,
        )
        return event_cls(self.even_data)

In [31]:
from abc import ABCMeta, abstractmethod, ABC


# class XMLEventParser(metaclass=ABCMeta):
#     @abstractmethod
#     def from_xml(xml_data: str):
#         """XML 형태 데이터 파싱"""
class XMLEventParser(metaclass=ABCMeta):
    @abstractmethod
    def from_xml(xml_data: str):
        """XML 형태 데이터 파싱"""


class JSONEventParser(metaclass=ABCMeta):
    @abstractmethod
    def from_json(json_data: str):
        """json 형태 데이터 파싱"""


class EventParser(XMLEventParser, JSONEventParser):
    def from_xml(xml_data: str):
        pass

    def from_json(json_data: str):
        pass


EventParser()

In [ ]:
# Interface segregation

from abc import ABC, ABCMeta, abstractmethod


class JSONEventParser_1(metaclass=ABCMeta):
    @abstractmethod
    def from_json(json_data: str):
        """json 형태 데이터 파싱"""


class JSONEventParser_2(ABC):
    @abstractmethod
    def from_json(json_data: str):
        """json 형태 데이터 파싱"""